### Importing Libraries

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd

### Defining the paths

In [ ]:
# Paths to the folders created for storing the interediate images
main_path='G:\\Projects\\soil_spectrogram\\sampleimages'
grey_path='G:\\Projects\\soil_spectrogram\\grey_img'
binary_path='G:\\Projects\\soil_spectrogram\\binary_img'
csv_path='G:\\Projects\\soil_spectrogram\\csv'
binary_uncrp='G:\\Projects\\soil_spectrogram\\binary_uncrp_img'

### Defining functions

In [ ]:
# Finding the upper bound of the spectrum
def upper_x(img):
    for i in range(img.shape[0]):
        summ=np.sum(img[i,:])
        if(summ>255*8):
            break
    return i
# Finding the lower bound of the spectrum
def lower_x(img):
    for i in range(img.shape[0]):
        summ=np.sum(img[img.shape[0] - i-1 :])
        if(summ>255*8):
            break
    return i

In [ ]:
final_ts=[]    
for image_path in sorted(os.listdir(main_path)):
    path=main_path+ '\\' + image_path
    print(path)    
    img=cv2.imread(path)
    print(img.shape)
    #Cropping values found out manually
    crp_img=img[500:800,445:1360] 
    gray = cv2.cvtColor(crp_img, cv2.COLOR_BGR2GRAY)
    #Converting to binary
    ret,thresh_img = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    #Saving to a binary images folder
    cv2.imwrite(binary_uncrp+'\\'+image_path, thresh_img) 
    x_upper_lim=upper_x(thresh_img)
    x_lower_lim=lower_x(thresh_img)
    #print(x_lim)
    #fin_image=gray[x_lim-5:x_lim+35,:] #Manually found out this width of spectrum according to the data-set
    
    fin_image=gray[x_upper_lim - 5 : gray.shape[0] - x_lower_lim + 5 , :]
    #Flipped the images since the spectrum was needed from Red to blue
    fin_image=np.fliplr(fin_image)
    reta,thres_img = cv2.threshold(fin_image,10,255,cv2.THRESH_BINARY)
    cv2.imwrite(grey_path+'\\'+image_path, fin_image)
    cv2.imwrite(binary_path+'\\'+image_path, thres_img)
    values=[]
    for i in range(fin_image.shape[1]):
        # Averaging out the image pixels column wise to get the the intensity values at each x co-ordinate
        avg=np.mean(fin_image[:,i])
        values.append(avg)
    # 915 is the number of pixels in x direction
    fvalues=np.reshape(np.array(values),(915,1))
    transposed_values=fvalues.transpose()
    final_ts.append(transposed_values)
    index=np.reshape(np.arange(0,915,1),(915,1))
    wavelength=np.reshape(np.arange(400, 1086.25, 0.75),(915,1))
    vals=np.concatenate((index,wavelength, fvalues), axis=1)
    fin_data=pd.DataFrame(data=vals, columns=['INDEX','LAMBDA','INTENSITY'])
    csv_paths=image_path.replace('jpg','csv')
    fin_data.to_csv(csv_path+'\\'+csv_paths, index=False)

In [ ]:
final_ts=np.concatenate(final_ts)

### A view of what the code is doing

![alt text](https://i.imgur.com/0tLUngi.gif "Visualization")